<a href="https://colab.research.google.com/github/marawayne/git-test/blob/master/tf_auto_gas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install seaborn

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

# Install TensorFlow
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

# TensorFlow and tf.keras
import tensorflow as tf

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from tensorflow import keras
from tensorflow.keras import layers

#check versions
print("TensorFlow ", tf.__version__)
print("Keras: ", keras.__version__)

In [0]:
#Get Data
dataset_path = keras.utils.get_file("auto-mpg.data", "https://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data")

In [0]:
column_names = ["MPG", "Cylinders", "Displacement", "Horsepower", "Weight", "Acceleration", "Model Year", "Origin"]
raw_dataset = pd.read_csv(dataset_path, names=column_names, na_values = "?", comment="\t", sep=" ", skipinitialspace=True)

dataset = raw_dataset.copy()
dataset.head()

In [0]:
dataset.isna().sum()

In [0]:
dataset = dataset.dropna()

In [0]:
origin = dataset.pop("Origin")

In [0]:
dataset["USA"] = (origin==1)*1.0
dataset["Europe"] = (origin==2)*1.0
dataset["Japan"] = (origin==3)*1.0
dataset.tail()


In [0]:
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)

In [0]:
sns.pairplot(train_dataset[["MPG","Cylinders","Displacement","Weight"]], diag_kind="kde")

In [0]:
train_stats = train_dataset.describe()
train_stats.pop("MPG")
train_stats = train_stats.transpose()
train_stats

In [0]:
#Split features from Labels
train_labels = train_dataset.pop("MPG")
test_labels = test_dataset.pop("MPG")

In [0]:
#Normalize the data

def norm(x):
  return (x - train_stats['mean']) / train_stats['std']

normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

##The Model


In [0]:
def build_model():
  model = keras.Sequential([
      layers.Dense(64, activation=tf.nn.relu, input_shape=[len(train_dataset.keys())]),
      layers.Dense(64, activation=tf.nn.relu),
      layers.Dense(1)
  ])
  
  optimizer = tf.keras.optimizers.RMSprop(0.001)
  
  model.compile(loss='mse',
               optimizer=optimizer,
               metrics=['mae', 'mse'])
  return model

In [0]:
model = build_model()

In [0]:
model.summary()

In [0]:
#try before train the model to see if it produces results
example_batch = normed_train_data[:10]
example_result = model.predict(example_batch)
example_result

In [0]:
#Train the model - Display training model progress by printing dot
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: 
      print('')
    print('.', end='')
    
EPOCHS = 1000


history = model.fit( normed_train_data, train_labels,
                   epochs=EPOCHS, validation_split = 0.2, verbose=0,
                   callbacks=[PrintDot()])      
      
      


In [0]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [0]:
import matplotlib.pyplot as plt

test_predictions = model.predict(normed_test_data).flatten()

plt.scatter(test_labels, test_predictions)
plt.xlabel('True Value [MPG]')
plt.ylabel('Predictions [MPG]')
plt.axis('equal')
plt.axis('square')

plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])
_=plt.plot([-100, 100], [-100, 100])
  